Descarga de Archivos

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

#URL de la página principal
url = 'https://siguealcongreso.org/monitoreo/sintema/lxiii/'

#Directorio donde se guardarán los archivos PDF
output_dir = 'documentos_pdf'
os.makedirs(output_dir, exist_ok=True)

#Obtener el contenido de la página
response = requests.get(url)
response.raise_for_status()
soup = BeautifulSoup(response.text, 'html.parser')

#Encontrar todos los enlaces "Leer documento"
links = soup.find_all('a', text='Leer documento')

#Descargar cada archivo PDF
for i, link in enumerate(links, start=1):
    pdf_url = link['href']
    pdf_response = requests.get(pdf_url)
    pdf_response.raise_for_status()
    
    #Nombre del archivo PDF basado en el número de secuencia
    pdf_filename = f'documento_{i}.pdf'
    pdf_path = os.path.join(output_dir, pdf_filename)
    
    #Guardar el archivo PDF
    with open(pdf_path, 'wb') as pdf_file:
        pdf_file.write(pdf_response.content)
    
    print(f'Descargado: {pdf_filename}')


Generación de Excel con hash SHA-256

In [ ]:
import os
import hashlib
import tkinter as tk
from tkinter import filedialog, messagebox
from PyPDF2 import PdfReader
from openpyxl import Workbook

#Función para calcular el hash SHA-256 del contenido del archivo PDF
def hash_pdf(file_path):
    hasher = hashlib.sha256()
    with open(file_path, 'rb') as f:
        pdf_reader = PdfReader(f)
        for page_num in range(len(pdf_reader.pages)):
            page_text = pdf_reader.pages[page_num].extract_text()
            if page_text:
                hasher.update(page_text.encode('utf-8'))
    return hasher.hexdigest()

#Función para cargar una carpeta y obtener los hashes de los archivos PDF
def load_directory_and_hash():
    root = tk.Tk()
    root.withdraw()  #Oculta la ventana principal de tkinter

    directory_path = filedialog.askdirectory()
    if directory_path:
        pdf_files = [f for f in os.listdir(directory_path) if f.endswith('.pdf')]
        if not pdf_files:
            messagebox.showwarning("Sin archivos PDF", "El directorio seleccionado no contiene archivos PDF.")
            return
        
        wb = Workbook()
        ws = wb.active
        ws.title = "Hashes PDF"

        #Añadir encabezados a la hoja de cálculo
        ws.append(["Nombre del Archivo", "Hash SHA-256"])

        #Iterar sobre los archivos PDF en la carpeta y calcular los hashes
        for pdf_file in pdf_files:
            file_path = os.path.join(directory_path, pdf_file)
            hash_value = hash_pdf(file_path)
            ws.append([pdf_file, hash_value])

        #Pedir al usuario que seleccione dónde guardar el archivo Excel
        output_file = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx")])
        if output_file:
            wb.save(output_file)
            messagebox.showinfo("Guardado Exitoso", f"Se han guardado los hashes en:\n{output_file}")
        else:
            messagebox.showwarning("Guardado Cancelado", "No se ha seleccionado un archivo para guardar. La operación ha sido cancelada.")

#Función para mostrar la ventana de carga de carpeta
def show_load_directory_window():
    load_directory_and_hash()

#Mostrar la ventana para cargar una carpeta y calcular los hashes
show_load_directory_window()

Aplicación

In [2]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox
import hashlib
from openpyxl import load_workbook
from PyPDF2 import PdfReader

#Función para calcular el hash SHA-256 del contenido del archivo PDF
def calcular_hash_pdf(ruta_archivo):
    hasher = hashlib.sha256()
    with open(ruta_archivo, 'rb') as f:
        lector_pdf = PdfReader(f)
        for num_pagina in range(len(lector_pdf.pages)):
            texto_pagina = lector_pdf.pages[num_pagina].extract_text()
            if texto_pagina:
                hasher.update(texto_pagina.encode('utf-8'))
    return hasher.digest()

#Función para abrir el archivo PDF
def abrir_archivo(ruta_archivo):
    os.startfile(ruta_archivo)

#Función para cargar una carpeta y verificar los hashes de los archivos PDF
def cargar_carpeta_y_verificar():
    root = tk.Tk()
    root.withdraw()  

    ruta_carpeta = filedialog.askdirectory(title="Seleccionar carpeta con archivos PDF")
    if not ruta_carpeta:
        messagebox.showerror("Error", "No se ha seleccionado una carpeta con archivos PDF.")
        root.destroy()
        return

    archivo_excel = filedialog.askopenfilename(title="Seleccionar archivo Excel", filetypes=[("Excel files", "*.xlsx")])
    if not archivo_excel:
        messagebox.showerror("Error", "No se ha seleccionado un archivo Excel.")
        root.destroy()
        return

    try:
        wb = load_workbook(archivo_excel)
        hoja = wb.active

        archivos_validos = []
        archivos_invalidos = []

        for nombre_archivo in os.listdir(ruta_carpeta):
            if nombre_archivo.endswith('.pdf'):
                ruta_archivo = os.path.join(ruta_carpeta, nombre_archivo)
                hash_calculado = calcular_hash_pdf(ruta_archivo)
                hash_encontrado = False

                for fila in hoja.iter_rows(min_row=2):
                    hash_del_excel = bytes.fromhex(fila[1].value)  #Convertir el hash de hexadecimal a bytes
                    if hash_calculado == hash_del_excel:
                        hash_encontrado = True
                        break

                if hash_encontrado:
                    archivos_validos.append(nombre_archivo)
                else:
                    archivos_invalidos.append((nombre_archivo, ruta_archivo))

        #Mostrar el resultado en una nueva ventana
        ventana_resultados = tk.Toplevel()
        ventana_resultados.title("Resultados de Verificación")
        ventana_resultados.geometry("700x400")
        ventana_resultados.resizable(False, False)

        marco_resultados = tk.Frame(ventana_resultados)
        marco_resultados.pack(fill=tk.BOTH, expand=True)

        canvas = tk.Canvas(marco_resultados)
        scrollbar = tk.Scrollbar(marco_resultados, orient="vertical", command=canvas.yview)
        marco_scrollable = tk.Frame(canvas)

        marco_scrollable.bind(
            "<Configure>",
            lambda e: canvas.configure(
                scrollregion=canvas.bbox("all")
            )
        )

        canvas.create_window((0, 0), window=marco_scrollable, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)

        for nombre_archivo in archivos_validos:
            tk.Label(marco_scrollable, text=f"{nombre_archivo} - Firma Válida", fg="green", font=("Helvetica", 10)).pack(anchor="w", pady=2)

        for nombre_archivo, ruta_archivo in archivos_invalidos:
            label = tk.Label(marco_scrollable, text=f"{nombre_archivo} - Firma Inválida", fg="red", font=("Helvetica", 10), cursor="hand2")
            label.pack(anchor="w", pady=2)
            label.bind("<Button-1>", lambda e, ruta=ruta_archivo: abrir_archivo(ruta))

        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")

        def cerrar_ventana_resultados():
            ventana_resultados.destroy()
            root.quit()

        ventana_resultados.protocol("WM_DELETE_WINDOW", cerrar_ventana_resultados)
        ventana_resultados.mainloop()

    except Exception as e:
        messagebox.showerror("Error", f"Error al leer el archivo Excel: {e}")
        root.destroy()
    finally:
        root.destroy()  #Cerrar la ventana principal de tkinter y terminar el programa

#Función para mostrar la ventana de bienvenida y empezar la verificación
def mostrar_ventana_bienvenida():
    ventana_bienvenida = tk.Tk()
    ventana_bienvenida.title("Bienvenido")
    ventana_bienvenida.geometry("700x350")
    ventana_bienvenida.resizable(False, False)
    tk.Label(ventana_bienvenida, text="Seleccione la carpeta con los documentos que desea verificar y el archivo Excel con los hashes originales:", font=("Helvetica", 10)).pack(pady=20)

    #Botón
    boton_seleccionar_carpeta = tk.Button(ventana_bienvenida, text="Seleccionar Carpeta", command=cargar_carpeta_y_verificar)
    boton_seleccionar_carpeta.config(height=3, width=50, font=("Helvetica", 12))
    boton_seleccionar_carpeta.pack(pady=10)

    def cerrar_ventana_bienvenida():
        ventana_bienvenida.destroy()
        ventana_bienvenida.quit()

    ventana_bienvenida.protocol("WM_DELETE_WINDOW", cerrar_ventana_bienvenida)

    ventana_bienvenida.mainloop()

# Mostrar la ventana de bienvenida
mostrar_ventana_bienvenida()